In [4]:
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
from tqdm import tqdm_notebook as tqdm
from joblib import Parallel, delayed
import time

In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('raw.csv')
print(df)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


        YearStart  YearEnd LocationAbbr          LocationDesc  \
0            2015     2015           AK                Alaska   
1            2015     2015           AL               Alabama   
2            2015     2015           AR              Arkansas   
3            2015     2015           AZ               Arizona   
4            2015     2015           CA            California   
5            2015     2015           CO              Colorado   
6            2015     2015           CT           Connecticut   
7            2015     2015           DC  District of Columbia   
8            2015     2015           FL               Florida   
9            2015     2015           GA               Georgia   
10           2015     2015           GU                  Guam   
11           2015     2015           HI                Hawaii   
12           2015     2015           IA                  Iowa   
13           2015     2015           ID                 Idaho   
14           2015     201

[403984 rows x 21 columns]


In [8]:
print(df.head)

<bound method NDFrame.head of         YearStart  YearEnd LocationAbbr          LocationDesc  \
0            2015     2015           AK                Alaska   
1            2015     2015           AL               Alabama   
2            2015     2015           AR              Arkansas   
3            2015     2015           AZ               Arizona   
4            2015     2015           CA            California   
5            2015     2015           CO              Colorado   
6            2015     2015           CT           Connecticut   
7            2015     2015           DC  District of Columbia   
8            2015     2015           FL               Florida   
9            2015     2015           GA               Georgia   
10           2015     2015           GU                  Guam   
11           2015     2015           HI                Hawaii   
12           2015     2015           IA                  Iowa   
13           2015     2015           ID                 Idah

[403984 rows x 21 columns]>


In [9]:
#description of year start/end count, confidence limit min and max
print(df.describe())

           YearStart        YearEnd  LowConfidenceLimit  HighConfidenceLimit  \
count  403984.000000  403984.000000       246819.000000        246819.000000   
mean     2012.729851    2012.756698           49.729946            62.654180   
std         1.546388       1.515176           83.633664            95.980311   
min      2001.000000    2001.000000            0.200000             0.420000   
25%      2011.000000    2012.000000           13.000000            20.200000   
50%      2013.000000    2013.000000           31.100000            45.600000   
75%      2014.000000    2014.000000           57.400000            72.200000   
max      2016.000000    2016.000000         1293.900000          2088.000000   

          LocationID  
count  403984.000000  
mean       31.003374  
std        17.700259  
min         1.000000  
25%        17.000000  
50%        30.000000  
75%        45.000000  
max        78.000000  


In [16]:
#Number of NaNs in the dataframe
#count_nan = len(df) - df.count()
#print(count_nan)
df.isnull().sum(axis = 0)

YearStart                         0
YearEnd                           0
LocationAbbr                      0
LocationDesc                      0
DataSource                        0
Topic                             0
Question                          0
DataValueUnit                 29865
DataValueType                     0
DataValue                    106167
LowConfidenceLimit           157165
HighConfidenceLimit          157165
StratificationCategory1           0
Stratification1                   0
GeoLocation                    2568
LocationID                        0
TopicID                           0
QuestionID                        0
DataValueTypeID                   0
StratificationCategoryID1         0
StratificationID1                 0
dtype: int64

In [19]:
df1= df.dropna()

In [20]:
df1.isnull().sum(axis = 0)

YearStart                    0
YearEnd                      0
LocationAbbr                 0
LocationDesc                 0
DataSource                   0
Topic                        0
Question                     0
DataValueUnit                0
DataValueType                0
DataValue                    0
LowConfidenceLimit           0
HighConfidenceLimit          0
StratificationCategory1      0
Stratification1              0
GeoLocation                  0
LocationID                   0
TopicID                      0
QuestionID                   0
DataValueTypeID              0
StratificationCategoryID1    0
StratificationID1            0
dtype: int64

In [22]:
df1.head(10)

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,DataValueUnit,DataValueType,DataValue,...,HighConfidenceLimit,StratificationCategory1,Stratification1,GeoLocation,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1
0,2015,2015,AK,Alaska,YRBSS,Alcohol,Alcohol use among youth,%,Crude Prevalence,22,...,24.5,Overall,Overall,"(64.84507995700051, -147.72205903599973)",2,ALC,ALC1_1,CrdPrev,OVERALL,OVR
1,2015,2015,AL,Alabama,YRBSS,Alcohol,Alcohol use among youth,%,Crude Prevalence,30.7,...,34.3,Overall,Overall,"(32.84057112200048, -86.63186076199969)",1,ALC,ALC1_1,CrdPrev,OVERALL,OVR
2,2015,2015,AR,Arkansas,YRBSS,Alcohol,Alcohol use among youth,%,Crude Prevalence,27.6,...,30.8,Overall,Overall,"(34.74865012400045, -92.27449074299966)",5,ALC,ALC1_1,CrdPrev,OVERALL,OVR
3,2015,2015,AZ,Arizona,YRBSS,Alcohol,Alcohol use among youth,%,Crude Prevalence,34.8,...,40.2,Overall,Overall,"(34.865970280000454, -111.76381127699972)",4,ALC,ALC1_1,CrdPrev,OVERALL,OVR
4,2015,2015,CA,California,YRBSS,Alcohol,Alcohol use among youth,%,Crude Prevalence,28.9,...,34.7,Overall,Overall,"(37.63864012300047, -120.99999953799971)",6,ALC,ALC1_1,CrdPrev,OVERALL,OVR
6,2015,2015,CT,Connecticut,YRBSS,Alcohol,Alcohol use among youth,%,Crude Prevalence,30.2,...,33.4,Overall,Overall,"(41.56266102000046, -72.64984095199964)",9,ALC,ALC1_1,CrdPrev,OVERALL,OVR
7,2015,2015,DC,District of Columbia,YRBSS,Alcohol,Alcohol use among youth,%,Crude Prevalence,20.1,...,21.0,Overall,Overall,"(38.907192, -77.036871)",11,ALC,ALC1_1,CrdPrev,OVERALL,OVR
8,2015,2015,FL,Florida,YRBSS,Alcohol,Alcohol use among youth,%,Crude Prevalence,33,...,35.0,Overall,Overall,"(28.932040377000476, -81.92896053899966)",12,ALC,ALC1_1,CrdPrev,OVERALL,OVR
10,2015,2015,GU,Guam,YRBSS,Alcohol,Alcohol use among youth,%,Crude Prevalence,25.3,...,29.2,Overall,Overall,"(13.444304, 144.793731)",66,ALC,ALC1_1,CrdPrev,OVERALL,OVR
11,2015,2015,HI,Hawaii,YRBSS,Alcohol,Alcohol use among youth,%,Crude Prevalence,25.1,...,27.2,Overall,Overall,"(21.304850435000446, -157.85774940299973)",15,ALC,ALC1_1,CrdPrev,OVERALL,OVR


In [23]:
print(df1.describe())

           YearStart        YearEnd  LowConfidenceLimit  HighConfidenceLimit  \
count  244210.000000  244210.000000       244210.000000        244210.000000   
mean     2012.794947    2012.823521           49.816000            62.850041   
std         1.515056       1.480388           83.839554            96.264711   
min      2009.000000    2010.000000            0.200000             0.500000   
25%      2012.000000    2012.000000           13.100000            20.400000   
50%      2013.000000    2013.000000           31.100000            45.700000   
75%      2014.000000    2014.000000           57.400000            72.300000   
max      2015.000000    2015.000000         1293.900000          2088.000000   

          LocationID  
count  244210.000000  
mean       29.634610  
std        16.470872  
min         1.000000  
25%        16.000000  
50%        29.000000  
75%        42.000000  
max        72.000000  
